## 3.4 Assignment

the task is to implement a dfs for the alpha cliques

in dolphins.txt the first line is the number of nodes 62
the following lines are starts and end vertices for undirected edges

The other file is dolphininfo.txt where the i:th line tells the name and sex
(F=female, M=male, U=unspecified) of dolphin (node) number i.


$ deg_S(v)  $   is the degree of v in set S
$ deg_V(v)  $   is the degree of v in the entire graph


a set S is an $ \alpha-clique $   if the smallest degree node in S, divided by the number of nodes - 1, is $ \geq \alpha $

### Pruning

even though $ \alpha-cliequedness $ is not monotonic, it is still possible to prune all supersets according to the formula. 


In [409]:
import numpy as np


alpha = 0.6

In [410]:
def read_graph(file_path):
    adjacency_list = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()
        # Extract the number of nodes from the first line
        first_line = lines[0].strip()
        num_nodes = int(first_line.split()[0])
        
        # Initialize adjacency list with empty lists for each node
        for node in range(1, num_nodes + 1):
            adjacency_list[node] = []
        
        # Parse each edge and populate the adjacency list
        for line in lines[1:]:
            parts = line.strip().split()
            if len(parts) == 2:
                u, v = int(parts[0]), int(parts[1])
                adjacency_list[u].append(v)
                adjacency_list[v].append(u)  # Because the graph is undirected
            else:
                raise Exception('wrong number')
    return adjacency_list


In [411]:
# Read the graph from the file
graph_toys = read_graph('toyset.txt')
graph_dolphins = read_graph('dolphins.txt')
print(len(graph_toys))

# Print the adjacency list
for node in graph_toys:
    print(f"{node}: {graph_toys[node]}")


5
1: [2, 4]
2: [1, 3, 4]
3: [2, 4, 5]
4: [1, 2, 3]
5: [3]


In [412]:
def calculate_degrees(graph, set_of_nodes):
    """
    calculates the degrees for a set of nodes, from the whole graph
    """
    all_degrees = {node: 0 for node in set_of_nodes}
    for node in set_of_nodes: 
        all_degrees[node] = len([neighbor for neighbor in graph[node]])
    return all_degrees

In [413]:
def is_alpha_clique(graph, set_of_nodes, alpha: float, constraint = 3):
    mindegrees = {node: 0 for node in set_of_nodes}

    S = len(set_of_nodes)
    if S < constraint:
        return False
    for node in set_of_nodes: 
        mindegrees[node] = len([neighbor for neighbor in graph[node] if neighbor in set_of_nodes])
    condition = min(mindegrees.values()) / (S - 1) >= alpha
    return condition



In [414]:

def is_limited_by_theorem(graph, set_of_nodes, alpha: float, constraint = 3):
    mindegrees = {node: 0 for node in set_of_nodes}

    S = len(set_of_nodes)
    if S < constraint:
        return False
    for node in set_of_nodes: 
        mindegrees[node] = len([neighbor for neighbor in graph[node] if neighbor in set_of_nodes])
    function_S = min(mindegrees.values()) / (S - 1)
    # the condition holds for the largest degree in the set S, considering all nodes around it


    all_degrees = calculate_degrees(graph, set_of_nodes)
    max_deg = max(all_degrees.values())
    upper_bound = 1  - (max_deg * (1 - alpha)) / ((S - 1)  *alpha)
    condition = function_S < upper_bound
    return condition


In [415]:
import numpy as np
from typing import Dict, List, Set, Tuple



def dfs(

    at: int,
    visited: np.ndarray,
    graph: Dict[int, List[int]],
    alpha,
    global_setof_cliques: Set[Tuple[int, ...]],
    counter: Dict[str, int],
    initial_call: bool = True,
    debug: bool = False, 
    constraint = 3
    ) -> None:
        
    """
    Performs a recursive Depth-First Search (DFS) starting from the given node.

    This function explores all possible paths in the graph by allowing backtracking.
    Each recursive call receives its own copy of the visited array to prevent
    interference between different exploration paths.

    Args:
        at (int): The current node being visited.
        visited (np.ndarray): A boolean array indicating which nodes have been visited.
        graph (Dict[int, List[int]]): The adjacency list representation of the graph.
        debug (bool): If True, prints debug information. Defaults to False.

    Returns:
        None
    """
    if debug and initial_call:
        print(f"ALGO Begins for Node {at}")

    # If the node is already visited in the current path, skip it

    # if debug:
    #     print(f"Visiting node {at}.")

    # Create a copy of the visited array for the current path
    visited = visited.copy()
    visited[at - 1] = True
    if debug:
        print(f"Node {at} marked as visited. Updated visited: {visited}")
    
    visited_nodes = tuple(sorted([i + 1 for i, value in enumerate(visited) if value]))
    if debug:
        print(f"\nTRYING OUT SET {visited_nodes}")
    
    if is_limited_by_theorem(graph=graph, set_of_nodes=visited_nodes, alpha=alpha, constraint = constraint):
        if debug:
            print("----------------------------------------")
            print(f" THEOREM STOP -- BACKTRACKING FROM {at}")
            print("----------------------------------------")
        return 

    # Increment the counter each time is_alpha_clique is called
    counter['tested_candidates'] += 1
    if is_alpha_clique(graph=graph, set_of_nodes=visited_nodes, alpha=alpha, constraint = constraint): # Writing .add stops duplicate entries from entering
        global_setof_cliques.add(visited_nodes)

    # Retrieve neighbors; use get to handle nodes with no neighbors
    neighbours = graph.get(at, [])
    if debug:
        print(f"\nNode {at} has neighbors: {neighbours}")

    for neighbour in neighbours:
        neighbour_is_visited = visited[neighbour - 1]
        if debug:
            print(f"Node {at} -> Attempting to visit neighbor {neighbour}", end='')
            if neighbour_is_visited:
                print("==> already visited")
            else:
                print("==> accepted")


        if not neighbour_is_visited:
            dfs(neighbour, visited, graph, alpha=alpha, global_setof_cliques=global_setof_cliques, initial_call=False, debug = debug, counter=counter)

    if debug:
        print(f"All neighbors of node {at} have been processed. Backtracking from node {at}.\n")



## 3.4 b) Improved DFS Search

To trim as much possible space, it is ideal to increase the RHS of the theorems upperbound equation, i.e, have as high of a degree as possible for the node. 

Thus when finding the the next node to test, the neighbor with the highest degree should be picked first. 

In [416]:

def dfs_improved(
    at: int,
    visited: np.ndarray,
    graph: Dict[int, List[int]],
    alpha,
    global_setof_cliques: Set[Tuple[int, ...]],
    counter: Dict[str, int],
    initial_call: bool = True,
    debug: bool = False,
    constraint = 3
) -> None:
    """
    Performs a recursive Depth-First Search (DFS) starting from the given node.
    
    Args:
        at (int): The current node being visited.
        visited (np.ndarray): A boolean array indicating which nodes have been visited.
        graph (Dict[int, List[int]]): The adjacency list representation of the graph.
        alpha: The alpha parameter for clique checking.
        global_setof_cliques (Set[Tuple[int, ...]]): A set to store found cliques.
        counter (Dict[str, int]): A dictionary to count tested candidates.
        initial_call (bool): Indicates if this is the initial call of DFS.
        debug (bool): If True, prints debug information.
    
    Returns:
        None
    """
    if debug and initial_call:
        print(f"ALGO Begins for Node {at}")

    # Create a copy of the visited array for the current path
    visited = visited.copy()
    visited[at - 1] = True
    if debug:
        print(f"Node {at} marked as visited. Updated visited: {visited}")
    
    visited_nodes = tuple(sorted([i + 1 for i, value in enumerate(visited) if value]))
    if debug:
        print(f"\nTRYING OUT SET {visited_nodes}")
    
    if is_limited_by_theorem(graph=graph, set_of_nodes=visited_nodes, alpha=alpha, constraint = constraint ):
        if debug:
            print("----------------------------------------")
            print(f" THEOREM STOP -- BACKTRACKING FROM {at}")
            print("----------------------------------------")
        return 
    
    # Increment the counter each time is_alpha_clique is called
    counter['tested_candidates'] += 1
    if is_alpha_clique(graph=graph, set_of_nodes=visited_nodes, alpha=alpha, constraint = constraint ):
        global_setof_cliques.add(visited_nodes)

    # Retrieve neighbors; use get to handle nodes with no neighbors
    neighbours = graph.get(at, [])
    if debug:
        print(f"\nNode {at} has neighbors: {neighbours}")

    neighbours_degrees = dict(zip(neighbours, calculate_degrees(graph, set_of_nodes=neighbours)))
    # Sort neighbors by degree in descending order
    sorted_neighbours = sorted(neighbours, key=lambda n: neighbours_degrees[n], reverse=True)

    for neighbour in sorted_neighbours:
        neighbour_is_visited = visited[neighbour - 1]
        if debug:
            print(f"Node {at} -> Attempting to visit neighbor {neighbour}", end='')
            if neighbour_is_visited:
                print("==> already visited")
            else:
                print("==> accepted")

        if not neighbour_is_visited:
            dfs_improved(
                neighbour,
                visited,
                graph,
                alpha=alpha,
                global_setof_cliques=global_setof_cliques,
                counter=counter,
                initial_call=False,
                debug=debug
            )

    if debug:
        print(f"All neighbors of node {at} have been processed. Backtracking from node {at}.\n")

In [417]:
def run_dfs(graph, alpha, algorithm = 'dfs_improved', constraint = 3):
    # Initialize the visited array with all False (unvisited)
    initial_visited: np.ndarray = np.zeros(len(graph), dtype=bool)
    global_setof_cliques: Set[Tuple[int, ...]] = set()
    counter = {'tested_candidates': 0}


    if algorithm == 'dfs_improved':
        for node_index in graph.keys():
            dfs_improved(node_index, initial_visited, graph=graph, alpha=alpha, global_setof_cliques=global_setof_cliques, counter = counter, initial_call=True, debug=False, constraint = constraint)
    elif algorithm == 'dfs':
        for node_index in graph.keys():
            dfs(node_index, initial_visited, graph=graph, alpha=alpha, global_setof_cliques=global_setof_cliques, initial_call=True, debug=False, counter = counter, constraint = constraint)
    else:
        raise Exception("wrong mode")

    print(f"Total tested candidates: {counter['tested_candidates']}")

    return global_setof_cliques


In [418]:
run_dfs(graph_toys, alpha=0.6, algorithm='dfs')

Total tested candidates: 57


{(1, 2, 3, 4), (1, 2, 4), (2, 3, 4)}

In [419]:

run_dfs(graph_dolphins, alpha=0.8, algorithm='dfs')

Total tested candidates: 33522


{(1, 11, 43),
 (1, 11, 43, 48),
 (1, 11, 48),
 (1, 15, 41),
 (1, 16, 41),
 (1, 43, 48),
 (2, 18, 28),
 (2, 20, 55),
 (2, 27, 28),
 (2, 42, 55),
 (3, 11, 43),
 (4, 9, 60),
 (6, 10, 14),
 (6, 10, 14, 58),
 (6, 10, 58),
 (6, 14, 58),
 (7, 10, 14),
 (7, 10, 14, 18),
 (7, 10, 14, 18, 58),
 (7, 10, 14, 42, 55, 58),
 (7, 10, 14, 58),
 (7, 10, 18),
 (7, 10, 18, 58),
 (7, 10, 58),
 (7, 14, 18),
 (7, 14, 18, 58),
 (7, 14, 55),
 (7, 14, 55, 58),
 (7, 14, 58),
 (7, 18, 58),
 (7, 55, 58),
 (8, 20, 31),
 (8, 20, 55),
 (9, 21, 29),
 (9, 38, 46),
 (9, 46, 60),
 (10, 14, 18),
 (10, 14, 18, 58),
 (10, 14, 33),
 (10, 14, 42),
 (10, 14, 42, 58),
 (10, 14, 58),
 (10, 18, 58),
 (10, 42, 58),
 (11, 43, 48),
 (14, 18, 58),
 (14, 42, 55),
 (14, 42, 55, 58),
 (14, 42, 58),
 (14, 55, 58),
 (15, 17, 34),
 (15, 17, 34, 38),
 (15, 17, 34, 38, 39, 44),
 (15, 17, 34, 39),
 (15, 17, 34, 51),
 (15, 17, 38),
 (15, 17, 39),
 (15, 17, 51),
 (15, 34, 35),
 (15, 34, 35, 38),
 (15, 34, 38),
 (15, 34, 38, 41),
 (15, 34, 38, 4

In [422]:

run_dfs(graph_dolphins, alpha=0.8, algorithm='dfs_improved', constraint=8)

Total tested candidates: 33522


{(1, 11, 43),
 (1, 11, 43, 48),
 (1, 11, 48),
 (1, 15, 41),
 (1, 16, 41),
 (1, 43, 48),
 (2, 18, 28),
 (2, 20, 55),
 (2, 27, 28),
 (2, 42, 55),
 (3, 11, 43),
 (4, 9, 60),
 (6, 10, 14),
 (6, 10, 14, 58),
 (6, 10, 58),
 (6, 14, 58),
 (7, 10, 14),
 (7, 10, 14, 18),
 (7, 10, 14, 18, 58),
 (7, 10, 14, 42, 55, 58),
 (7, 10, 14, 58),
 (7, 10, 18),
 (7, 10, 18, 58),
 (7, 10, 58),
 (7, 14, 18),
 (7, 14, 18, 58),
 (7, 14, 55),
 (7, 14, 55, 58),
 (7, 14, 58),
 (7, 18, 58),
 (7, 55, 58),
 (8, 20, 31),
 (8, 20, 55),
 (9, 21, 29),
 (9, 38, 46),
 (9, 46, 60),
 (10, 14, 18),
 (10, 14, 18, 58),
 (10, 14, 33),
 (10, 14, 42),
 (10, 14, 42, 58),
 (10, 14, 58),
 (10, 18, 58),
 (10, 42, 58),
 (11, 43, 48),
 (14, 18, 58),
 (14, 42, 55),
 (14, 42, 55, 58),
 (14, 42, 58),
 (14, 55, 58),
 (15, 17, 34),
 (15, 17, 34, 38),
 (15, 17, 34, 38, 39, 44),
 (15, 17, 34, 39),
 (15, 17, 34, 51),
 (15, 17, 38),
 (15, 17, 39),
 (15, 17, 51),
 (15, 34, 35),
 (15, 34, 35, 38),
 (15, 34, 38),
 (15, 34, 38, 41),
 (15, 34, 38, 4

## 3.4 c) Maximal $\alpha-cliques $


a maximal alpha clique is a clique, that cannot be extended by adding a new node (dolphin)

to find the maximal alpha cliques, using set size >= 5:
1. sort all cliques
2. starting from smallest, check if those values exist in other sets
3. prune those supersets

In [421]:

alpha_cliques = run_dfs(graph_dolphins, alpha=0.8, algorithm='dfs_improved', constraint=5)


Total tested candidates: 33522
